In [28]:
import ollama
import json
import re
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

print('importattu')

TOPIC = 'Tekoäly'
model = 'hf.co/tensorblock/LumiOpen_Llama-Poro-2-8B-Instruct-GGUF'

# set up vector database and the retriever:
PERSIST_DIR = "./index/chroma"
COLLECTION = "artificial_intelligence"
EMBED = "intfloat/multilingual-e5-small"
emb = HuggingFaceEmbeddings(model_name=EMBED)
vs = Chroma(
    collection_name=COLLECTION,
    persist_directory=PERSIST_DIR,
    embedding_function=emb,
)
retriever = vs.as_retriever(search_kwargs={"k": 6})

print('vector store ja retriever rdy')

# get information from RAG
def fetchDocuments(user_query):
    docs = retriever.get_relevant_documents(user_query)
    context = "RAG konteksti: " + "\n\n---\n\n".join(
        f"[LÄHDE: {d.metadata.get('source','tuntematon')}"
        + (f", sivu {d.metadata.get('page')}" if d.metadata.get('page') is not None else "")
        + f"]\n{d.page_content[:800]}"
        for d in docs
    )
    
    return "CONTEXT (faktoihin nojaamiseen, ei käyttäjälle näytettäväksi):\n" + context

def parse_router_plan(text: str) -> dict:
    # poista koodiaidat jos niitä on
    t = text.strip()
    if t.startswith("```"):
        t = re.sub(r"^```[a-zA-Z]*\n?", "", t)
        t = re.sub(r"\n?```$", "", t).strip()

    # yritä suoraan
    try:
        return json.loads(t)
    except json.JSONDecodeError:
        pass

    # regex: etsi ensimmäinen {...} lohko
    m = re.search(r"\{.*\}", t, flags=re.S)
    if m:
        try:
            return json.loads(m.group(0))
        except json.JSONDecodeError:
            pass

    # fallback
    return {"route": "suora", "RAG": "no", "reason": "fallback: invalid json"}

print('starting...')

# Starting comment
beginning_preprompt = f"opetat oppilasta aiheesta {TOPIC}. Kirjoita alkuviesti, jossa kerrot \
käsiteltävän aiheen ja kysy oppilaalta kysymys, joka selvittää hieman hänen lähtötasoaan (esim. \
Onko tämä aihe sinulle entuudestaan yhtään tuttu?)"

pedagogical_prompt = f"Käyttäen mahdollista RAG kontekstia alla sekä omia tietojasi, \
ja vastaa näiden pohjalta käyttäjän kysymykseen." 


router_message = 'Sinun tehtäväsi on valita miten ohjelmaa suoritetaan seuraavaksi käyttäjäviestin \
perusteella. Jos viestistä löytyy seuraavia sanoja "lähde, lähdeaineisto, viimeaikainen, uusi, \
etsiä, etsi, viittaus, viitaten", tai jos vaikuttaa siltä, että käyttäjä etsii jotain spesifiä \
tietoa, niin valitse vastauspohjaan kohtaan RAG "yes" ja aseta route-muuttujaan arvo "epäsuora". \
reason kohtaan kirjoita miksi valitsit sen reitin (esim. koska joku edellä mainituista sanoista \
oli käytetty tai viestissä pyydettiin jotain spesifiä tietoa.)\
Anna vastauksesi seuraavan vastauspohjan \
muodossa: {"route":"suora |epäsuora", "reason":"miksi valitsit tämän reitin", "RAG" : "yes | no"} \
Palauta VAIN JSON-yksiö ilman selityksiä, ilman koodiaitoja, ilman ylimääräisiä rivinvaihtoja:'

resp = ollama.chat(model=model, messages=[{'role': 'system', 'content': beginning_preprompt}])
print('Viola: ', resp['message']['content'])

# Conversation loop
messages = []
while True:
    
    user = input('Sinä: ')

    #exit condition met?
    if user in [':exit', ':e', ':quit', ':q', ':lopeta', ':L']:
        break

    messages.append({"role": "user", "content": user})
   
    # router - decide what to do next
    router_resp = ollama.chat(model=model, messages=[
        {'role': 'system', 'content': router_message},
        {'role': 'user', 'content': user}
    ])

    router_content = router_resp["message"]["content"]
    json_resp = parse_router_plan(router_content)

    #decide if tools are needed:
    rag_context = ""
    if json_resp["RAG"] == "yes":
        # get most relevant documents from the vector database:
        rag_context = fetchDocuments(user)
    print(rag_context)

    #combine pedagogical_prompt and tool-based prompts
    system_prompt = pedagogical_prompt + '\n\n' + rag_context
    print('\nsystem prompt: ', system_prompt) 
    # produce the answer:
    resp = ollama.chat(model=model, messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user}
    ])
    messages.append(resp['message'])
    resp_msg = resp['message']['content']
    print('vastaus saatu')
    # print the answer:
    print('\nViola: ', resp_msg)
    messages



importattu
vector store ja retriever rdy
starting...


ResponseError: model requires more system memory (5.6 GiB) than is available (5.5 GiB) (status code: 500)

['qwer']
